# Seasonal SBAS Stacks

SBAS time series analyses require a coherent, fully-connected, redundant stack of interferograms with small perpendicular and temporal baselines.

Seasonal impacts to scatterers often break coherence, making it difficult to prepare long time series for MintPy. In these cases, it may possible to create a seasonal stack, removing seasons that would cause a loss of coherence. 

To avoid creating a asfnnected stack, seasonal gaps can be bridged with interferograms having one-year temporal baselines.

---
## Define seasonal SBAS stack parameters

In [ ]:
# This does not need to be the first scene in the stack. 
# slc_id = 'S1A_IW_SLC__1SDV_20200519T004428_20200519T004455_032627_03C760_27C2-SLC'
slc_id = 'S1_008794_IW2_20200519T004446_VV_27C2-BURST'

# mm-dd
season_start_date = '4-15'
season_end_date = '10-15'

# yyyy-mm-dd
stack_start_date = '2020-01-01'
stack_end_date = '2021-11-01'

max_temporal_baseline = 36
max_perp_baseline = 150

# Helps determine the expected number of pairs for a given temporal baseline threshold when bridging seasons
repeat_pass_freq = 12

---
## Build an SBASStack object

- If the `season`, `start`, `end`, and `refSceneName` args are passed upon initialization, an SBAS stack will be immediately calculated using default repeat pass frequency and baselines
- Parameters can be added or adjusted, generating updated SBAS stacks

Baseline and repeat pass frequency defaults:
- `perpendicularBaseline`: 400
- `temporBaseline`: 36
- `repeatPassFrequency`: 12

In [ ]:
import asf_search as asf

args = {
    'season': (season_start_date, season_end_date), 
    'start': stack_start_date, 
    'end': stack_end_date, 
    'refSceneName': slc_id,
    # 'perpendicularBaseline': 400,
    # 'temporBaseline': 36,
    # 'repeatPassFrequency': 12
}

sbas = asf.ASFSBASStack(**args)

In [ ]:
sbas.sbas_stack

In [ ]:
sbas.plot()

---
## Update baselines and re-plot

In [ ]:
sbas.perp_baseline = 150
sbas.temporal_baseline = 48
sbas.plot()

---
## Adjust the time bounds and re-plot

In [ ]:
sbas.temporal_baseline = 36
sbas.end = '2023-11-01'
sbas.plot()

---
## Create a list of scene IDs for each InSAR pair to submit as HyP3 On-Demand Processing jobs

In [ ]:
insar_pairs = sbas.get_insar_pairs()
print(f'InSAR Pairs: {len(insar_pairs)}')

---
## Authenticate with HyP3 using your EarthData Login Credentials

In [ ]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3(prompt=True)

---
## Define a HyP3 project name for your SBAS stack

In [ ]:
project_name = input("Enter a HyP3 project name")

---
## Request the interferograms

In [ ]:
job_type = 'INSAR_GAMMA' if 'SLC' in slc_id else 'INSAR_ISCE_BURST'

insar_jobs = sdk.Batch()
for pair in insar_pairs:
    if 'SLC' in slc_id:
        insar_jobs += hyp3.submit_insar_job(pair[0], pair[1], name=project_name)
    else:
        insar_jobs += hyp3.submit_insar_isce_burst_job(pair[0], pair[1], name=project_name)
print(insar_jobs)

In [ ]:
insar_jobs = hyp3.watch(insar_jobs)